In [1]:
import os
from pydub import AudioSegment
import numpy as np

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input, concatenate, Dropout, SpatialDropout1D, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, LearningRateScheduler

from sklearn.model_selection import train_test_split
from clr_callback import CyclicLR
from qrnn import QRNN

C:\Users\Ranet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Ranet\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (thean

## Raw data preprocessing:

In [2]:
MAP_DIR_PATH = 'MAPS/PROTOTYPE MAPS/'
MAP_NAME = 'combined_maps'

In [3]:
# Load the beatmap hitcirlces

with open(os.path.join(MAP_DIR_PATH, MAP_NAME + '.txt')) as f:
    content = [','.join(x.strip().split(',')[:3]) for x in f.readlines()]

for i in range(10):
    print(content[i])

80,64,107
128,192,380
160,192,516
176,320,789
208,320,925
232,192,1198
264,192,1334
288,320,1607
320,320,1743
336,192,2016


In [4]:
joined_maps = '\n'.join(content)

length = 140
sequences = list()
for i in range(length, len(joined_maps)):
    # select sequence of tokens
    seq = joined_maps[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

for i in range(5):
    print(sequences[i])
    print('------------')

Total Sequences: 4284542
80,64,107
128,192,380
160,192,516
176,320,789
208,320,925
232,192,1198
264,192,1334
288,320,1607
320,320,1743
336,192,2016
360,192,2152
384,1
------------
0,64,107
128,192,380
160,192,516
176,320,789
208,320,925
232,192,1198
264,192,1334
288,320,1607
320,320,1743
336,192,2016
360,192,2152
384,19
------------
,64,107
128,192,380
160,192,516
176,320,789
208,320,925
232,192,1198
264,192,1334
288,320,1607
320,320,1743
336,192,2016
360,192,2152
384,192
------------
64,107
128,192,380
160,192,516
176,320,789
208,320,925
232,192,1198
264,192,1334
288,320,1607
320,320,1743
336,192,2016
360,192,2152
384,192,
------------
4,107
128,192,380
160,192,516
176,320,789
208,320,925
232,192,1198
264,192,1334
288,320,1607
320,320,1743
336,192,2016
360,192,2152
384,192,2
------------


In [5]:
chars = sorted(list(set(joined_maps)))
mapping = dict((c, i) for i, c in enumerate(chars))

sequences_mapped = list()
for seq in sequences:
    # integer encode line
    encoded_seq = [mapping[char] for char in seq]
    # store
    sequences_mapped.append(encoded_seq)

for i in range(5):
    print(sequences_mapped[i])
    print('------------')
    


# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

[10, 2, 1, 8, 6, 1, 3, 2, 9, 0, 3, 4, 10, 1, 3, 11, 4, 1, 5, 10, 2, 0, 3, 8, 2, 1, 3, 11, 4, 1, 7, 3, 8, 0, 3, 9, 8, 1, 5, 4, 2, 1, 9, 10, 11, 0, 4, 2, 10, 1, 5, 4, 2, 1, 11, 4, 7, 0, 4, 5, 4, 1, 3, 11, 4, 1, 3, 3, 11, 10, 0, 4, 8, 6, 1, 3, 11, 4, 1, 3, 5, 5, 6, 0, 4, 10, 10, 1, 5, 4, 2, 1, 3, 8, 2, 9, 0, 5, 4, 2, 1, 5, 4, 2, 1, 3, 9, 6, 5, 0, 5, 5, 8, 1, 3, 11, 4, 1, 4, 2, 3, 8, 0, 5, 8, 2, 1, 3, 11, 4, 1, 4, 3, 7, 4, 0, 5, 10, 6, 1, 3]
------------
[2, 1, 8, 6, 1, 3, 2, 9, 0, 3, 4, 10, 1, 3, 11, 4, 1, 5, 10, 2, 0, 3, 8, 2, 1, 3, 11, 4, 1, 7, 3, 8, 0, 3, 9, 8, 1, 5, 4, 2, 1, 9, 10, 11, 0, 4, 2, 10, 1, 5, 4, 2, 1, 11, 4, 7, 0, 4, 5, 4, 1, 3, 11, 4, 1, 3, 3, 11, 10, 0, 4, 8, 6, 1, 3, 11, 4, 1, 3, 5, 5, 6, 0, 4, 10, 10, 1, 5, 4, 2, 1, 3, 8, 2, 9, 0, 5, 4, 2, 1, 5, 4, 2, 1, 3, 9, 6, 5, 0, 5, 5, 8, 1, 3, 11, 4, 1, 4, 2, 3, 8, 0, 5, 8, 2, 1, 3, 11, 4, 1, 4, 3, 7, 4, 0, 5, 10, 6, 1, 3, 11]
------------
[1, 8, 6, 1, 3, 2, 9, 0, 3, 4, 10, 1, 3, 11, 4, 1, 5, 10, 2, 0, 3, 8, 2, 1, 3, 11, 4, 1, 7

In [6]:
sequences = np.array(sequences_mapped)
X, y = sequences[:10000,:-1], sequences[:10000,-1]

print(X[2])
print(y[2])

#X = X.reshape(X.shape[0], X.shape[1], 1)
#sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
#X = np.array(sequences)
#y = to_categorical(y, num_classes=vocab_size)

print(X[2])
print(y[2])

[ 1  8  6  1  3  2  9  0  3  4 10  1  3 11  4  1  5 10  2  0  3  8  2  1
  3 11  4  1  7  3  8  0  3  9  8  1  5  4  2  1  9 10 11  0  4  2 10  1
  5  4  2  1 11  4  7  0  4  5  4  1  3 11  4  1  3  3 11 10  0  4  8  6
  1  3 11  4  1  3  5  5  6  0  4 10 10  1  5  4  2  1  3  8  2  9  0  5
  4  2  1  5  4  2  1  3  9  6  5  0  5  5  8  1  3 11  4  1  4  2  3  8
  0  5  8  2  1  3 11  4  1  4  3  7  4  0  5 10  6  1  3 11]
4
[ 1  8  6  1  3  2  9  0  3  4 10  1  3 11  4  1  5 10  2  0  3  8  2  1
  3 11  4  1  7  3  8  0  3  9  8  1  5  4  2  1  9 10 11  0  4  2 10  1
  5  4  2  1 11  4  7  0  4  5  4  1  3 11  4  1  3  3 11 10  0  4  8  6
  1  3 11  4  1  3  5  5  6  0  4 10 10  1  5  4  2  1  3  8  2  9  0  5
  4  2  1  5  4  2  1  3  9  6  5  0  5  5  8  1  3 11  4  1  4  2  3  8
  0  5  8  2  1  3 11  4  1  4  3  7  4  0  5 10  6  1  3 11]
4


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)
print(y_train.shape)

(8000, 140)
(8000,)


In [8]:
def step_decay(epoch):
    lrate = 0.001
    if epoch > 11:
        lrate == 0.0001

    return lrate

lrate = LearningRateScheduler(step_decay, verbose=1)

In [9]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 400, input_length=length))
model.add(QRNN(2500, window_size=3, dropout=0.1, return_sequences=True))#, 
               #kernel_regularizer=l2(1e-4), bias_regularizer=l2(1e-4), 
               #kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10)))
model.add(QRNN(2500, window_size=3, dropout=0.1, return_sequences=True))
model.add(QRNN(2500, window_size=3, dropout=0.1, return_sequences=True))
model.add(QRNN(2500, window_size=3, dropout=0.1))

model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 140, 400)          6000      
_________________________________________________________________
qrnn_1 (QRNN)                (None, 140, 2500)         9007500   
_________________________________________________________________
qrnn_2 (QRNN)                (None, 140, 2500)         56257500  
_________________________________________________________________
qrnn_3 (QRNN)                (None, 140, 2500)         56257500  
_________________________________________________________________
qrnn_4 (QRNN)                (None, 2500)              56257500  
_________________________________________________________________
dense_1 (Dense)              (None, 15)                37515     
Total params: 177,823,515
Trainable params: 177,823,515
Non-trainable params: 0
______________________________________________________________

In [11]:
# compile model
#clr = CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_train, y_train, batch_size=15, epochs=1000,
           validation_data=(X_val, y_val), verbose=1, callbacks=[lrate])

Train on 8000 samples, validate on 2000 samples
Epoch 1/1000

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.


ResourceExhaustedError: OOM when allocating tensor with shape[3,1,2500,7500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_1/Adam/mul_38 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam_1/beta_2/read, training_1/Adam/Variable_18/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training_1/Adam/gradients/loss_1/dense_1_loss/clip_by_value/Minimum_grad/Reshape/_281 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1722_...ad/Reshape", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions

In [ ]:
print(model.optimizer.lr)
import keras.backend as K

K.set_value(model.optimizer.lr, 0.0001)


In [ ]:
#model.load_weights('MODEL/no_music_test_weights_2.h5')

model.fit(X_train, y_train, batch_size=512, epochs=10,
           validation_data=(X_val, y_val), verbose=1)#, callbacks=[clr])

In [ ]:
from keras.models import load_model
model = load_model('MODEL/no_music_test_model_embedding.h5')

In [ ]:
model.save('MODEL/no_music_test_model_embedding.h5')
model.save_weights('MODEL/no_music_test_weights_embedding.h5')
np.save('MODEL/no_music_test_embedding.np', model.get_weights())

In [ ]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        #encoded = encoded.reshape(encoded.shape[0], encoded.shape[1], 1)
        # one hot encode
       # encoded = encoded, num_classes=len(mapping)
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text


seed_text = '''
317,197,281
256,241,612
357,346,778
396,208,944
336,268,1109
195,197,1275
258,0,1607
374,80,1773
256,149,1938
292,74,2104
180,103,2270
101,259,2601
40,194,2933
203,238,3099
126,350,3265
298,265,3596
255,322,3928
336,200,4093
'''

print(generate_seq(model, mapping, length, seed_text, 1000))